Information about YAKE!: https://github.com/LIAAD/yake

The lower the score, the more relevant the keyword is.

In [1]:
"""
To-do:

 [+] Add preprocessing that Michael created
 [+] Create a csv/json file that has ticket_id and tags assigned to it
 [+] Do analysis with one word only (not two)
 
"""

import yake
import json
import re
import operator
import csv

## Get data from csv and put it into the dictionary

In [2]:
csv_file_path = 'Data/tickets_clean.csv'
json_file_path = 'data/ticket_data.json'

data = []
with open(csv_file_path, encoding="utf8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for rows in csv_reader:
        ticket = {}
        
        ticket["id"] = rows['id']
        ticket["title"] = rows['title']
        ticket["content"] = rows['content']
        
        data.append(ticket)
        
with open(json_file_path, 'w') as json_file:
    json_file.write(json.dumps(data))

## Function that implements YAKE!

In [3]:
def get_keywords(text):
    
    language = "en"
    max_ngram_size = 2
    deduplication_thresold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    
    return keywords

## Get keywords, create 2 files: file with ticket id and it's tags and file that has a set of teh most common tags

In [4]:
"""
for each ticket in the json file:
    - Combine title and content
    - Get tags
    - Add to the dict
"""
tags_dict = {}
tickets_with_tags = {}

for i in range(20000):
    text = data[i]['title'] + ' ' + data[i]['content']
    
    #print(text)
    try:
        tags_list = get_keywords(text)
    except:
        print('An error occurred.')
        continue
    #print(tags_list)
    
    tags_for_ticket = []
    for tag_value_pair in tags_list:
        tag = tag_value_pair[0]
        value = tag_value_pair[1]
         
        if value <= 0.1:
            tags_for_ticket.append(tag)
            
            if tag in tags_dict:
                tags_dict[tag] += 1
            else:
                tags_dict[tag] = 1
            
    
    tickets_with_tags[data[i]['id']] = tags_for_ticket

#print(tickets_with_tags)


In [ ]:
for i in range(20000):
    text = data[i]['title'] + ' ' + data[i]['content']
    
    #print(text)
    try:
        tags_list = get_keywords(text)
    except:
        print('An error occurred.')
        continue
    #print(tags_list)
    
    tags_for_ticket = []
    for tag_value_pair in tags_list:
        tag = tag_value_pair[0]
        value = tag_value_pair[1]
         
        if value <= 0.1:
            tags_for_ticket.append(tag)
            
            if tag in tags_dict:
                tags_dict[tag] += 1
            else:
                tags_dict[tag] = 1
            
    
    tickets_with_tags[data[i]['id']] = tags_for_ticket

In [ ]:
with open('Two words/YAKE_tickets-tags_two_words.json', 'w') as file:
    file.write(json.dumps(tickets_with_tags))

## Show all data 

In [ ]:
sorted_list = sorted(tags_dict.items(), key=operator.itemgetter(1), reverse = True)

print("{:<35} {:<7}".format('tag', 'count'))

for k, v in sorted_list:
    print("{:<35} {:<7}".format(k, v))

## Show the most useful tags only

In [ ]:
with open('Two words/YAKE_ALL_tags_two_words.txt', 'w') as file:
    file.write(json.dumps(tags_dict))
    

usefull_tag_dict = {}

for key, value in tags_dict.items():
    if value > 10:
        usefull_tag_dict[key] = value

print(len(usefull_tag_dict))
        
with open('Two words/YAKE_tags_two_words.txt', 'w') as file:
    file.write(json.dumps(usefull_tag_dict))

In [ ]:
sorted_list = sorted(usefull_tag_dict.items(), key=operator.itemgetter(1), reverse = True)

print("{:<35} {:<7}".format('tag', 'count'))

for k, v in sorted_list:
    print("{:<35} {:<7}".format(k, v))